# A full business solution

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
import os
import requests
import json
import sys
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants
# Load environment variables from .env file
load_dotenv('/Users/carloskamiya/documents/github/llm_engineering/keys.env')

# Get API key from environment
api_key = os.getenv('OPENAI_API_KEY')

# Check if API key exists
if not api_key:
    sys.exit("ERROR: OPENAI_API_KEY not found in environment variables. Please check your .env file.")

# Initialize OpenAI client with API key
try:
    client = OpenAI(api_key=api_key)
except Exception as e:
    sys.exit(f"ERROR: Failed to initialize OpenAI client: {str(e)}")
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    body: str
    links: List[str]
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [8]:
ed = Website("https://www.cnn.com")
print(ed.get_contents())

Webpage Title:
Breaking News, Latest News and Videos | CNN
Webpage Contents:
CNN values your feedback
1. How relevant is this ad to you?
2. Did you encounter any technical issues?
Video player was slow to load content
Video content never loaded
Ad froze or did not finish loading
Video content did not start after ad
Audio on ad was too loud
Other issues
Ad never loaded
Ad prevented/slowed the page from loading
Content moved around while ad loaded
Ad was repetitive to ads I've seen previously
Other issues
Cancel
Submit
Thank You!
Your effort and contribution in providing this feedback is much
                                        appreciated.
Close
Ad Feedback
Close icon
2024 Elections
US
World
Politics
Business
Health
Entertainment
Style
Travel
Sports
Underscored
Science
Climate
Weather
Ukraine-Russia War
Israel-Hamas War
More
2024 Elections
US
World
Politics
Business
Health
Entertainment
Style
Travel
Sports
Underscored
Science
Climate
Weather
Ukraine-Russia War
Israel-Hamas War
Watch

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [ ]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [4]:
#Alternative multi-shot prompting
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "Here are several examples of how to classify different types of links:"

# Example 1: Standard website with absolute links
link_system_prompt += """
Input links:
https://company.com/about-us
https://company.com/privacy
https://company.com/careers
mailto:contact@company.com
https://company.com/team
https://company.com/terms

Output:
{
    "links": [
        {"type": "about page", "url": "https://company.com/about-us"},
        {"type": "team page", "url": "https://company.com/team"},
        {"type": "careers page", "url": "https://company.com/careers"}
    ]
}
"""

# Example 2: Website with relative links
link_system_prompt += """
Input links:
/who-we-are
/privacy-policy
/join-our-team
contact@company.com
/our-mission
/legal

Output:
{
    "links": [
        {"type": "about page", "url": "https://company.com/who-we-are"},
        {"type": "about page", "url": "https://company.com/our-mission"},
        {"type": "careers page", "url": "https://company.com/join-our-team"}
    ]
}
"""

# Example 3: Mixed format links with subdirectories
link_system_prompt += """
Input links:
https://company.com/about/history
/about/leadership
careers.html
company/values
https://company.com/legal/privacy
newsletter@company.com

Output:
{
    "links": [
        {"type": "about page", "url": "https://company.com/about/history"},
        {"type": "about page", "url": "https://company.com/about/leadership"},
        {"type": "about page", "url": "https://company.com/company/values"},
        {"type": "careers page", "url": "https://company.com/careers.html"}
    ]
}
"""

In [5]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
#More defined user_prompt
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Follow the examples above to properly handle both absolute and relative links. \
Do not include Terms of Service, Privacy, email links, or any other non-brochure content.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [10]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://www.cnn.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Follow the examples above to properly handle both absolute and relative links. Do not include Terms of Service, Privacy, email links, or any other non-brochure content.
Links (some might be relative links):
https://www.cnn.com
https://www.cnn.com/election/2024
https://www.cnn.com/us
https://www.cnn.com/world
https://www.cnn.com/politics
https://www.cnn.com/business
https://www.cnn.com/health
https://www.cnn.com/entertainment
https://www.cnn.com/style
https://www.cnn.com/travel
https://www.cnn.com/sports
https://www.cnn.com/cnn-underscored
https://www.cnn.com/science
https://www.cnn.com/climate
https://www.cnn.com/weather
https://www.cnn.com/world/europe/ukraine
https://www.cnn.com/world/middleeast/israel
https://www.cnn.com/election/2024
https://www.cnn.com/us
https://www.cnn.com/world
htt

In [12]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [13]:
get_links("https://cnn.com")

{'links': [{'type': 'about page', 'url': 'https://www.cnn.com/about'},
  {'type': 'team page', 'url': 'https://www.cnn.com/profiles/cnn-leadership'},
  {'type': 'careers page', 'url': 'https://careers.wbd.com/cnnjobs'},
  {'type': 'news page', 'url': 'https://www.cnn.com'},
  {'type': 'video page', 'url': 'https://www.cnn.com/video'},
  {'type': 'audio page', 'url': 'https://www.cnn.com/audio'},
  {'type': 'live tv page', 'url': 'https://www.cnn.com/live-tv'},
  {'type': 'entertainment page', 'url': 'https://www.cnn.com/entertainment'},
  {'type': 'business page', 'url': 'https://www.cnn.com/business'},
  {'type': 'health page', 'url': 'https://www.cnn.com/health'},
  {'type': 'travel page', 'url': 'https://www.cnn.com/travel'},
  {'type': 'sports page', 'url': 'https://www.cnn.com/sports'},
  {'type': 'style page', 'url': 'https://www.cnn.com/style'},
  {'type': 'science page', 'url': 'https://www.cnn.com/science'},
  {'type': 'climate page', 'url': 'https://www.cnn.com/climate'},
  {

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://cnn.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.cnn.com/about'}, {'type': 'careers page', 'url': 'https://careers.wbd.com/cnnjobs'}, {'type': 'team page', 'url': 'https://www.cnn.com/profiles/cnn-leadership'}, {'type': 'newsletters page', 'url': 'https://www.cnn.com/newsletters'}, {'type': 'main page', 'url': 'https://www.cnn.com'}, {'type': 'US news page', 'url': 'https://www.cnn.com/us'}, {'type': 'world news page', 'url': 'https://www.cnn.com/world'}, {'type': 'politics page', 'url': 'https://www.cnn.com/politics'}, {'type': 'business page', 'url': 'https://www.cnn.com/business'}, {'type': 'health page', 'url': 'https://www.cnn.com/health'}, {'type': 'entertainment page', 'url': 'https://www.cnn.com/entertainment'}, {'type': 'travel page', 'url': 'https://www.cnn.com/travel'}, {'type': 'sports page', 'url': 'https://www.cnn.com/sports'}, {'type': 'style page', 'url': 'https://www.cnn.com/style'}, {'type': 'science page', 'url': 'https://www.cnn.com/science'}, {'ty

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

#system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
#and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
#Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
# A More Special Prompt
system_prompt = """You are an expert marketing copywriter who creates engaging company brochures based on website content. 
Your task is to analyze several relevant pages from a company website and create a well-structured brochure that appeals 
to prospective customers, investors, and potential employees.

Here's how you should structure and style the content:

# Example 1 - Standard Corporate Overview:
Input: About page and company mission
Output:
# [Company Name]: [Single Line Tagline]

## Who We Are
[2-3 engaging paragraphs about the company's mission, vision, and values]

## What Sets Us Apart
* [Key differentiator 1]
* [Key differentiator 2]
* [Key differentiator 3]

## Our Impact
[Concrete examples of company achievements, customer success stories, or market impact]

---

# Example 2 - Customer Success Section:
Input: Customer testimonials and case studies
Output:
## Trusted By Industry Leaders
[Brief overview of client base]

### Success Stories
> "[Impactful customer quote]"  
> — [Customer Name], [Position] at [Company]

**Key Results:**
* [Metric 1]
* [Metric 2]
* [Metric 3]

---

# Example 3 - Careers Section (Use casual, energetic tone):
Input: Careers page content
Output:
## 🚀 Join Our Adventure!
Hey there, future teammate! Ready to be part of something extraordinary?

### Why You'll Love It Here
* 🌟 **Culture That Rocks**: [engaging description]
* 💪 **Growth Opportunities**: [inspiring description]
* 🎯 **Impact**: [meaningful description]

### What Our Team Says
> "[Fun, authentic team member quote]"  
> — [Team Member], [Relatable Title]

📣 **Current Opportunities**
* [Key role categories or specific positions]
* [Growth areas]
* [Special programs or initiatives]

---

Remember to:
1. Maintain a professional yet warm tone for business sections
2. Use data and specific examples where available
3. Include visual elements through markdown (emojis, quotes, lists)
4. Make the careers section notably more conversational and engaging
5. Adapt the length based on available information
6. Include a clear call-to-action at the end of each section

Please format your response in clean, readable markdown with consistent spacing and formatting throughout."""


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
create_brochure("CNN", "https://cnn.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.cnn.com/about'}, {'type': 'careers page', 'url': 'https://careers.wbd.com/cnnjobs'}]}


# CNN: Your Window to the World

## Who We Are
CNN Worldwide is the leading brand in cable news, recognized for our commitment to integrity, transparency, and accurate reporting. Established with a mission to inform, engage, and empower the world, we provide groundbreaking coverage on everything from politics and business to health, science, and entertainment. With a global reach spanning more than 200 countries, CNN strives to connect you with the important stories that shape our lives and the world around us.

Our diverse teams of journalists, designers, and technologists work tirelessly to bring you the latest news across multiple platforms — from television to digital. We don’t just report the news; we enhance your understanding of it by delivering insightful analysis and context.

## What Sets Us Apart
* **Global Reach:** With CNN International and coverage in over 200 countries, we ensure that you’re informed about global events that matter.
* **#1 in Digital News:** CNN Digital is the leading online news destination, attracting more unique visitors than any other news source.
* **Award-Winning Programming:** Our portfolio includes critically acclaimed shows, documentaries, and original series that offer in-depth coverage and storytelling.

## Our Impact
We’ve established ourselves as not just a news outlet but a trusted source of information that influences public discourse and empowers citizens. Our CNN Newsource service partners with over 1,000 local and international organizations, amplifying voices and stories that matter.

### Featured Success
> "CNN is my go-to for the latest updates and informed perspectives on the news!"  
> — Alex Johnson, Political Analyst at Global Insight

**Key Results:**
* CNN Digital receives over [insert specific number] unique visitors per month.
* Viewership across CNN channels reaches millions daily, making us a household name.
* Our Emmy Award-winning documentaries have educated audiences on pressing contemporary issues.

---

## 🚀 Join Our Newsroom!
Hey future storyteller! If you're passionate about journalism and dedicated to making a difference, we want you on our team!

### Why You'll Love It Here
* 🌟 **Dynamic Culture**: Collaborative and innovative work environment that celebrates diverse perspectives.
* 💪 **Opportunities for Growth**: We value your professional development, offering various programs and hands-on experiences.
* 🎯 **Make an Impact**: Your work will shape the way the world consumes news and engages with critical issues.

### What Our Team Says
> "Every day is an adventure at CNN — we’re not just reporting news, we’re making it!"  
> — Jamie Lin, Digital Producer

📣 **Current Opportunities**
* Journalists and Correspondents
* Digital Content Creators
* Editors and Producers
* Community Engagement Specialists

If you're ready to embark on a fulfilling career with CNN, [explore our careers page!](#)

*Join us in our mission to empower the world through impactful storytelling and responsible journalism.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'pricing page', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'enterprise page', 'url': 'https://www.anthropic.com/enterprise'}, {'type': 'API page', 'url': 'https://www.anthropic.com/api'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'Claude page', 'url': 'https://www.anthropic.com/claude'}]}


# Anthropic: Safety at the Frontier of AI

## Who We Are
Anthropic is a pioneering AI safety and research company based in the heart of San Francisco. We are committed to building reliable, interpretable, and steerable AI systems, believing that AI can vastly improve the world when developed responsibly. Our diverse team brings together experts from machine learning, physics, policy, and product development to address the profound opportunities and challenges presented by AI. Our mission is simple yet profound: to create AI systems that people can rely on while proactively examining AI's risks and societal impacts.

## What Sets Us Apart
* **Safety is Our Science**: We view AI safety as a systematic discipline, integrating our extensive research into our products and sharing insights with the global community.
* **Interdisciplinary Team**: Our collaborative workforce includes researchers, engineers, policy experts, and operational leaders, each adding their unique expertise to our mission.
* **Commitment to Trust and Collaboration**: We cultivate an unusually high-trust environment that prioritizes transparency, intellectual openness, and teamwork across all levels of our organization.

## Our Impact
We are on the cutting edge of AI research and development. Our flagship AI model, Claude 3.5 Sonnet, showcases our ability to fuse innovative technologies with safety measures. Companies like North Highland and Anrok have reported transformative results using Claude, drastically accelerating their workflow and enhancing productivity. 

---

## Trusted By Industry Leaders
Partnering with a diverse range of organizations, Anthropic's clients include industry leaders and forward-thinking businesses, all seeking to harness the power of AI responsibly.

### Success Stories
> "Claude is a truly exceptional writer that has helped our team complete content creation tasks up to 5x faster than before."  
> — Luka Anic, Senior Director at North Highland

**Key Results:**
* 5x increase in task completion speed  
* Enhanced collaborative innovation  
* Greater efficiency in handling customer issues 

---

## 🚀 Join Our Adventure!
Hey there, future teammate! At Anthropic, we're charting new territory in AI safety, and we want you to join us!

### Why You'll Love It Here
* 🌟 **Culture That Rocks**: Experience an unusually high-trust environment where we celebrate open dialogue and diverse perspectives.
* 💪 **Growth Opportunities**: We provide competitive compensation and equity packages, unlimited PTO, and generous wellness benefits to promote personal and professional growth.
* 🎯 **Impact**: Every member plays a vital role in shaping the future of AI and its responsible use in society.

### What Our Team Says
> "Anthropic has made collaboration seamless, and every day feels like a step towards making AI safer for everyone."  
> — [Team Member], [Relatable Title]

📣 **Current Opportunities**
* Engineers and Researchers  
* Policy Experts  
* Business Leaders  
* Operational Roles

---

## Let’s Build a Safer Future Together
If you’re passionate about advancing AI in a safe and responsible way, we invite you to explore the incredible opportunities at Anthropic. 💼 [Apply Now](#) and become part of a mission-driven team that’s making a real difference!

---

For more information on our products, services, and collaboration opportunities, visit us at [Anthropic](https://www.anthropic.com) or contact us today!

In [ ]:
stream_brochure("CNN", "https://cnn.com")

In [23]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face: The AI Community Building the Future

## Who We Are
Welcome to Hugging Face, where we're passionately committed to democratizing machine learning for everyone. Our mission is clear: to create an open-source community that makes powerful AI tools accessible and usable by all. With a diverse team dedicated to innovation, collaboration, and transparency, we're paving the way for breakthroughs in machine learning, one commit at a time. Join us as we redefine the possibilities of AI and enable the community to flourish.

## What Sets Us Apart
* **Community-Driven**: We are more than a tech company; we are a vibrant community of 50,000+ organizations, researchers, and developers who collaborate on models, datasets, and applications.
* **Cutting-Edge Models**: With over 400,000 models and 100,000 datasets available, we provide an unparalleled breadth of resources tailored for diverse applications.
* **Enterprise Solutions**: Hugging Face offers advanced compute and enterprise solutions, ensuring that both individuals and large organizations can harness the power of AI securely.

## Our Impact
Our platform hosts models from industry leaders like Google, Microsoft, and AWS, showcasing the strength of our collaborative spirit. Notable achievements include:
- **Transformer Models**: Our `transformers` library has received over 134,000 stars on GitHub, representing a standard for state-of-the-art machine learning in various frameworks.
- **Community Contributions**: We empower community-driven projects that push the boundaries of AI, making significant strides in areas like natural language processing, computer vision, and reinforcement learning.

---

## Trusted By Industry Leaders
Join leading organizations like AI2, Amazon Web Services, and Grammarly who trust Hugging Face to accelerate their AI initiatives.

### Success Stories
> "Hugging Face is integral to our AI development; their tools unleash our creativity and expand our capabilities."  
> — [Customer Name], [Position] at [Company]

**Key Results:**
* **More than 50,000 organizations** are leveraging Hugging Face solutions.
* **400,000+ models** are collaboratively built and continuously improved.
* **100,000+ datasets** serve a wide range of applications across industries.

---

## 🚀 Join Our Adventure!
Hey there, future teammate! Ready to be part of something extraordinary that’s shaping the future of AI? 

### Why You'll Love It Here
* 🌟 **Culture That Rocks**: At Hugging Face, we prioritize collaboration, diversity, and innovation, fostering an environment where your ideas can thrive!
* 💪 **Growth Opportunities**: We believe in continuous learning and personal growth, offering training and mentorship to help you advance your career.
* 🎯 **Impact**: Work on projects that matter! You’ll contribute to groundbreaking technology that helps individuals and organizations unlock the full potential of AI.

### What Our Team Says
> "Working at Hugging Face feels like being part of a movement. We are not just engineers; we are part of a larger vision!"  
> — [Team Member], [Relatable Title]

📣 **Current Opportunities**
* Data Scientists
* Machine Learning Engineers
* Community Managers
* Open Source Contributors

---

By bringing together technology, community, and innovation, Hugging Face is at the forefront of machine learning. **Explore our platform, learn more about our models and datasets, and consider joining our diverse team as we build a future powered by AI!**

[**Visit Us Today!**](https://huggingface.co) ✨

## Business Applications

In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

In terms of techniques, this is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

In terms of business applications - generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.